# OLS, Testing

This notebook estimates a linear regression and tests various hypotheses using standard errors assuming iid residuals (Gauss-Markov assumptions).

You may also consider the [HypothesisTests.jl](https://github.com/JuliaStats/HypothesisTests.jl) package (not used here).

## Load Packages and Extra Functions

In [1]:
MyModulePath = joinpath(pwd(),"src")
!in(MyModulePath,LOAD_PATH) && push!(LOAD_PATH,MyModulePath)
using FinEcmt_OLS

In [2]:
#=
include(joinpath(pwd(),"src","FinEcmt_OLS.jl"))
using .FinEcmt_OLS
=#

In [3]:
using DelimitedFiles, Statistics, LinearAlgebra, Distributions

## Loading Data

In [4]:
x = readdlm("Data/FFmFactorsPs.csv",',',skipstart=1)

                #yearmonth, market, small minus big, high minus low
(ym,Rme,RSMB,RHML) = (x[:,1],x[:,2]/100,x[:,3]/100,x[:,4]/100)
x = nothing

printlnPs("Sample size:",size(Rme))

Sample size:    (388,)


## OLS under the Gauss-Markov Assumptions

(assuming iid residuals)

In [5]:
Y = Rme
T = size(Y,1)
X = [ones(T) RSMB RHML]

(b,u,_,V,R²) = OlsGM(Y,X)
std_iid = sqrt.(diag(V))

printblue("OLS Results (assuming iid residuals):\n")
xNames = ["c","SMB","HML"]
printmat(b,std_iid;colNames=["b","std_iid"],rowNames=xNames)

OLS Results (assuming iid residuals):

            b   std_iid
c       0.007     0.002
SMB     0.217     0.073
HML    -0.429     0.074



## Testing a Joint Hypothesis

Since the estimator $\hat{\beta}_{_{k\times1}}$ satisfies

$\hat{\beta}-\beta_{0} \sim N(0,V_{k\times k}),$

we can easily apply various tests. Consider a joint linear hypothesis of the
form

$H_0: R\beta=q,$

where $R$ is a $J \times k$ matrix and $q$ is a $J$-vector. To test this, use

$(R\beta-q)^{\prime}(RVR^{\prime}) ^{-1}(R\beta
-q)\overset{d}{\rightarrow}\chi_{J}^{2}.$

How we estimate $V$ depends on whether there is heteroskedasticity and/or autocorrelation (discussed below).

In [6]:
R = [0 1 0;               #testing if b₂=0 and b₃=0
     0 0 1]
q = [0;0]
test_stat = (R*b-q)'inv(R*V*R')*(R*b-q)    #R*V*R' is 2x2

printblue("Testing Rb = a:")
printmat([test_stat,quantile(Chisq(2),0.9)];rowNames=["test statistic","10% critical value"])

Testing Rb = a:
test statistic        60.010
10% critical value     4.605



## Regression Diagnostics: Testing All Slope Coefficients

The `OlsR2Test()` function tests all slope coefficients (or equivalently, the $R^2$) of a regression. Notice that the regression must contain an intercept for R² to be useful.

In [7]:
@doc2 OlsR2Test

```
OlsR2Test(R²,T,df)
```

Test of all slope coefficients. Notice that the regression must contain an intercept for R² to be useful.

### Input

  * `R²::Number`:    R² value
  * `T::Int`:        number of observations
  * `df::Number`:    number of (non-constant) regressors

### Output

  * `RegrStat::Number`: test statistic
  * `pval::Number`:     p-value


In [8]:
using CodeTracking
println(@code_string OlsR2Test(1.0,1,25))    #print the source code

function OlsR2Test(R²,T,df)
    RegrStat = T*R²/(1-R²)           #R\^2[TAB]
    pval     = ccdf(Chisq(df),RegrStat)    #same as 1-cdf()
    return RegrStat, pval
end


In [9]:
df = size(X,2) - 1              #number of slope coefficients
(RegrStat,pval) = OlsR2Test(R²,T,df)

printblue("Test of all slopes = 0:\n")
printmat([RegrStat,pval],rowNames=["stat","p-val"])

Test of all slopes = 0:

stat     60.165
p-val     0.000

